# Задача
1 — изучить датасет <br>
2 — предобработать датасет <br>
3 — выбрать модель: классика + нейронка <br>
4 — обучение, проверка, тестирование   
5 — Вывод </p>
ДО 7.12

In [47]:
import pandas as pd
import spacy

nlp = spacy.load("ru_core_news_sm")
nlp.Defaults.stop_words |= {"Александр", "Куприн", "Иван", "Бунин"} # Это оператор "Союза" (множеств, не нерушимых республик свободы)

## Создание датасета

In [ ]:
kuprin_files = {
    "./Kuprin/chudesniy_doktor.txt": "0",
    "./Kuprin/nochnaya_smena.txt": "0",
    "./Kuprin/beliy_pudel.txt": "0",
    "./Kuprin/poedinok.txt": "0",
    "./Kuprin/izumrud.txt": "0",
    "./Kuprin/granatovy_braslet.txt": "0",
    "./Kuprin/anaphema.txt": "0",        
    "./Kuprin/zvezda_solomona.txt": "0", 
    "./Kuprin/limonnaya_korka.txt": "0",
    "./Kuprin/sinaya_zvezda.txt": "0",
    }

bunin_files = {
    "./Bunin/alleys.txt": "1",
    "./Bunin/archive.txt": "1",
    "./Bunin/brothers.txt": "1",
    "./Bunin/dreams.txt": "1",
    "./Bunin/kostsy.txt": "1",
    "./Bunin/monday.txt": "1",
    "./Bunin/san-francisco.txt": "1",
    "./Bunin/various.txt": "1",
    }

def preprocess(file_path, bunin):
    with open(file_path, "r") as file:
        sentences = list(nlp(file.read()).sents)
        sentences = [" ".join([w.lemma_ for w in list(s) if not w.is_punct and not w.is_stop and w.text.isalnum()]) for s in sentences]
    return {"Sentence": sentences, "IsBunin": [bunin]*len(sentences)}
        
literator_data = pd.DataFrame()

for key, value in kuprin_files.items():
    literator_data = pd.concat([literator_data, pd.DataFrame(data=preprocess(key, value))])

for key, value in bunin_files.items():
    literator_data = pd.concat([literator_data, pd.DataFrame(data=preprocess(key, value))])

literator_data.head()

In [ ]:
literator_data.to_csv(path_or_buf="./literator_data.csv", index=None)

## Предобработка 
Используем токенайзер SpaCy для получения норм векторов предложений, а также отсекаем слишком короткие предложения для большей точности.

In [51]:
MIN_SENT_LENGTH = 10

data = pd.read_csv("./literator_data.csv")
data.dropna(inplace=True)

data["Sentence"] = data["Sentence"].map(lambda sentence: pd.NaT \
                                        if len(nlp(sentence)) < MIN_SENT_LENGTH \
                                        else nlp(sentence).vector_norm)

data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

data.head()

,Sentence,IsBunin
0,5.841487,0
1,6.270028,0
2,5.841487,0
3,6.203244,0
4,4.762572,0


In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6590 entries, 0 to 6589
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  6590 non-null   object
 1   IsBunin   6590 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 103.1+ KB


Мешаем датасет, т.к. изначально он шёл по порядку

In [53]:
from sklearn.utils import shuffle
data = shuffle(data)

## Обучаем нейросеть RandomForestClassifier
Эта архитектура нейросети позволяет хорошо справляться с задачей классификации за счёт универсальности, внимания к деталям и малой восприимчивости к шуму, а также может избегать переобучения за счёт построения нескольких независимых деревьев решений.

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import matplotlib.pylab as plt

TRAIN_TEST_DIVIDER = len(data) // 3 * 2
print(f"The border of division is: {TRAIN_TEST_DIVIDER}")

train_x = data.drop(["IsBunin"], axis=1)[:TRAIN_TEST_DIVIDER]
train_y = data.drop(["Sentence"], axis=1)[:TRAIN_TEST_DIVIDER]
test_x = data.drop(["IsBunin"], axis=1)[TRAIN_TEST_DIVIDER:]
test_y = data.drop(["Sentence"], axis=1)[TRAIN_TEST_DIVIDER:]

rf = RandomForestClassifier()
rf.fit(train_x, train_y)

y_pred = rf.predict(test_x)

print(f"Accuracy: {accuracy_score(test_y, y_pred)}")

The border of division is: 4392


/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.5150136487716106


In [58]:
test_line_kuprin = "Снизу, из машинного отделения, вместе с теплым запахом нефти, доносилось непрерывное шипение, мягкие удары работающих поршней и какие-то глубокие, правильные вздохи, в такт которым так же размеренно вздрагивала деревянная палуба «Ястреба»."
# Из рассказа А.И. Куприна "Одиночество"
print("Куприн" if rf.predict([[nlp(test_line_kuprin).vector_norm]])==[0] else "Бунин")

Бунин


/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Настройка гиперпараметров леса для большей точности

### Инициируем процесс подбора лучших гиперпараметров (подбираться они будут случайно)

In [59]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

param_dist = {"n_estimators": randint(50,500),
              "max_depth": randint(1,20)}

rf = RandomForestClassifier()       # "Обнуляем" модель

# 
rand_search = RandomizedSearchCV(rf,    
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

rand_search.fit(train_x, train_y)

/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f080fe1fa60>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f080fb2f550>})

In [60]:
best_rf = rand_search.best_estimator_

print(f"Best hyperparameters: {rand_search.best_params_}")

Best hyperparameters: {'max_depth': 2, 'n_estimators': 230}


In [61]:
y_pred = best_rf.predict(test_x)

print(f"Accuracy: {accuracy_score(test_y, y_pred)}")

Accuracy: 0.5809827115559599


In [64]:
test_line_kuprin = "Мой слуга, повар и спутник по охоте — полесовщик Ярмола вошел в комнату, согнувшись под вязанкой дров, сбросил ее с грохотом на пол и подышал на замерзшие пальцы."
# Из произведения А.И. Куприна "Олеся"
print("Куприн" if best_rf.predict([[nlp(test_line_kuprin).vector_norm]])==[0] else "Бунин")

Бунин


/home/magic_window/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Вывод: модель путает Бунина и Куприна. Впрочем, как и автор...